In [1]:
import keras
from keras.layers import Input
import sys
import os
sys.path.insert(0, "../models")
sys.path.insert(0, "../layers")
import models
import yaml
from train import *

Using TensorFlow backend.
/Users/sionisummers/miniconda3/envs/py3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/sionisummers/miniconda3/envs/py3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/sionisummers/miniconda3/envs/py3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/User

### Make a dummy options object, load the data

In [2]:
class dummy:
    def __init__(self):
        self.x = 0
options = dummy()
options.inputFile = '../data/processed-pythia82-lhc13-all-pt1-50k-r1_h022_e0175_t220_nonu_truth.z'
options.outputDir = 'binary_model_double_float_initialised'
os.mkdir(options.outputDir)
options.tree = 't_allpar_new'
yamlConfig = parse_config('train_config_threelayer_binary.yml')
X_train_val, X_test, y_train_val, y_test, labels  = get_features(options, yamlConfig)

Loading configuration from train_config_threelayer_binary.yml


/Users/sionisummers/Work/hls4ml/keras-training-clean/train/train.py:137: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(config)


(986806,)
('j_ptfrac', 'j_pt', 'j_eta', 'j_mass', 'j_tau1_b1', 'j_tau2_b1', 'j_tau3_b1', 'j_tau1_b2', 'j_tau2_b2', 'j_tau3_b2', 'j_tau32_b1', 'j_tau32_b2', 'j_zlogz', 'j_c1_b0', 'j_c1_b1', 'j_c1_b2', 'j_c2_b1', 'j_c2_b2', 'j_d2_b1', 'j_d2_b2', 'j_d2_a1_b1', 'j_d2_a1_b2', 'j_m2_b1', 'j_m2_b2', 'j_n2_b1', 'j_n2_b2', 'j_tau1_b1_mmdt', 'j_tau2_b1_mmdt', 'j_tau3_b1_mmdt', 'j_tau1_b2_mmdt', 'j_tau2_b2_mmdt', 'j_tau3_b2_mmdt', 'j_tau32_b1_mmdt', 'j_tau32_b2_mmdt', 'j_c1_b0_mmdt', 'j_c1_b1_mmdt', 'j_c1_b2_mmdt', 'j_c2_b1_mmdt', 'j_c2_b2_mmdt', 'j_d2_b1_mmdt', 'j_d2_b2_mmdt', 'j_d2_a1_b1_mmdt', 'j_d2_a1_b2_mmdt', 'j_m2_b1_mmdt', 'j_m2_b2_mmdt', 'j_n2_b1_mmdt', 'j_n2_b2_mmdt', 'j_mass_trim', 'j_mass_mmdt', 'j_mass_prun', 'j_mass_sdb2', 'j_mass_sdm1', 'j_multiplicity', 'j_g', 'j_q', 'j_w', 'j_z', 'j_t', 'j_undef')


### Load the models
Load the trained float model, and an unitialised binary model of the same architecture
Set the weights of the BNN to the trained float model weights

In [3]:
float_model = keras.models.load_model("float_model_double/KERAS_check_best_model.h5")
binary_model = models.three_layer_model_binary_relu_double(Input(shape=X_train_val.shape[1:]), y_train_val.shape[1], l1Reg=yamlConfig['L1Reg'])
ifc = 0
for il in range(len(float_model.layers) - 1):
    if 'fc' in float_model.layers[il+1].name:
        # Get the weights (ignore the biases)
        w = float_model.layers[il+1].get_weights()[0]
        binary_model.layers[il].set_weights([w])
keras_model = binary_model
print_model_to_json(keras_model,options.outputDir + '/' + 'KERAS_model.json')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [4]:
startlearningrate=0.0001
adam = Adam(lr=startlearningrate)
keras_model.compile(optimizer=adam, loss=[yamlConfig['KerasLoss']], metrics=['accuracy'])

In [5]:
callbacks=all_callbacks(stop_patience=1000,
                         lr_factor=0.5,
                         lr_patience=10,
                         lr_epsilon=0.000001,
                         lr_cooldown=2,
                         lr_minimum=0.0000001,
                         outputDir=options.outputDir)

/Users/sionisummers/miniconda3/envs/py3/lib/python3.6/site-packages/keras/callbacks/callbacks.py:998: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [6]:
 keras_model.fit(X_train_val, y_train_val, batch_size = 1024, epochs = 100,
                 validation_split = 0.25, shuffle = True, callbacks = callbacks.callbacks)

Train on 592083 samples, validate on 197361 samples
Epoch 1/100
592083/592083 [==============================] - 14s 24us/step - loss: 0.7271 - accuracy: 0.6782 - val_loss: 1.0454 - val_accuracy: 0.5811

***callbacks***
saving losses to binary_model_double_float_initialised/losses.log

Epoch 00001: val_loss improved from inf to 1.04538, saving model to binary_model_double_float_initialised/KERAS_check_best_model.h5

Epoch 00001: val_loss improved from inf to 1.04538, saving model to binary_model_double_float_initialised/KERAS_check_best_model_weights.h5

Epoch 00001: saving model to binary_model_double_float_initialised/KERAS_check_model_last.h5

Epoch 00001: saving model to binary_model_double_float_initialised/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 2/100
592083/592083 [==============================] - 18s 30us/step - loss: 0.6212 - accuracy: 0.6935 - val_loss: 0.7808 - val_accuracy: 0.6184

***callbacks***
saving losses to binary_model_double_float_initialised


***callbacks***
saving losses to binary_model_double_float_initialised/losses.log

Epoch 00013: val_loss improved from 0.40599 to 0.40338, saving model to binary_model_double_float_initialised/KERAS_check_best_model.h5

Epoch 00013: val_loss improved from 0.40599 to 0.40338, saving model to binary_model_double_float_initialised/KERAS_check_best_model_weights.h5

Epoch 00013: saving model to binary_model_double_float_initialised/KERAS_check_model_last.h5

Epoch 00013: saving model to binary_model_double_float_initialised/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 14/100
592083/592083 [==============================] - 13s 21us/step - loss: 0.3373 - accuracy: 0.7088 - val_loss: 0.4083 - val_accuracy: 0.6288

***callbacks***
saving losses to binary_model_double_float_initialised/losses.log

Epoch 00014: val_loss did not improve from 0.40338

Epoch 00014: val_loss did not improve from 0.40338

Epoch 00014: saving model to binary_model_double_float_initialised/KERAS_chec

592083/592083 [==============================] - 12s 20us/step - loss: 0.3286 - accuracy: 0.7135 - val_loss: 0.4316 - val_accuracy: 0.6070

***callbacks***
saving losses to binary_model_double_float_initialised/losses.log

Epoch 00027: val_loss did not improve from 0.35024

Epoch 00027: val_loss did not improve from 0.35024

Epoch 00027: saving model to binary_model_double_float_initialised/KERAS_check_model_last.h5

Epoch 00027: saving model to binary_model_double_float_initialised/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 28/100
592083/592083 [==============================] - 12s 21us/step - loss: 0.3282 - accuracy: 0.7134 - val_loss: 0.3709 - val_accuracy: 0.6733

***callbacks***
saving losses to binary_model_double_float_initialised/losses.log

Epoch 00028: val_loss did not improve from 0.35024

Epoch 00028: val_loss did not improve from 0.35024

Epoch 00028: saving model to binary_model_double_float_initialised/KERAS_check_model_last.h5

Epoch 00028: saving mo


***callbacks***
saving losses to binary_model_double_float_initialised/losses.log

Epoch 00041: val_loss did not improve from 0.35024

Epoch 00041: val_loss did not improve from 0.35024

Epoch 00041: saving model to binary_model_double_float_initialised/KERAS_check_model_last.h5

Epoch 00041: saving model to binary_model_double_float_initialised/KERAS_check_model_last_weights.h5

Epoch 00041: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-05.

***callbacks end***

Epoch 42/100
592083/592083 [==============================] - 14s 23us/step - loss: 0.3229 - accuracy: 0.7178 - val_loss: 0.4208 - val_accuracy: 0.6395

***callbacks***
saving losses to binary_model_double_float_initialised/losses.log

Epoch 00042: val_loss did not improve from 0.35024

Epoch 00042: val_loss did not improve from 0.35024

Epoch 00042: saving model to binary_model_double_float_initialised/KERAS_check_model_last.h5

Epoch 00042: saving model to binary_model_double_float_initialised/KERAS_check_m


***callbacks end***

Epoch 55/100
592083/592083 [==============================] - 15s 25us/step - loss: 0.3245 - accuracy: 0.7163 - val_loss: 0.4539 - val_accuracy: 0.6091

***callbacks***
saving losses to binary_model_double_float_initialised/losses.log

Epoch 00055: val_loss did not improve from 0.33411

Epoch 00055: val_loss did not improve from 0.33411

Epoch 00055: saving model to binary_model_double_float_initialised/KERAS_check_model_last.h5

Epoch 00055: saving model to binary_model_double_float_initialised/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 56/100
592083/592083 [==============================] - 16s 27us/step - loss: 0.3237 - accuracy: 0.7171 - val_loss: 0.3893 - val_accuracy: 0.6551

***callbacks***
saving losses to binary_model_double_float_initialised/losses.log

Epoch 00056: val_loss did not improve from 0.33411

Epoch 00056: val_loss did not improve from 0.33411

Epoch 00056: saving model to binary_model_double_float_initialised/KERAS_check_mo


Epoch 00069: val_loss improved from 0.33411 to 0.32296, saving model to binary_model_double_float_initialised/KERAS_check_best_model_weights.h5

Epoch 00069: saving model to binary_model_double_float_initialised/KERAS_check_model_last.h5

Epoch 00069: saving model to binary_model_double_float_initialised/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 70/100
592083/592083 [==============================] - 16s 26us/step - loss: 0.3211 - accuracy: 0.7189 - val_loss: 0.4281 - val_accuracy: 0.6228

***callbacks***
saving losses to binary_model_double_float_initialised/losses.log

Epoch 00070: val_loss did not improve from 0.32296

Epoch 00070: val_loss did not improve from 0.32296

Epoch 00070: saving model to binary_model_double_float_initialised/KERAS_check_model_last.h5

Epoch 00070: saving model to binary_model_double_float_initialised/KERAS_check_model_last_weights.h5

Epoch 00070: saving model to binary_model_double_float_initialised/KERAS_check_model_epoch70.h5

***c

592083/592083 [==============================] - 6s 10us/step - loss: 0.3212 - accuracy: 0.7188 - val_loss: 0.3441 - val_accuracy: 0.7091

***callbacks***
saving losses to binary_model_double_float_initialised/losses.log

Epoch 00084: val_loss did not improve from 0.32296

Epoch 00084: val_loss did not improve from 0.32296

Epoch 00084: saving model to binary_model_double_float_initialised/KERAS_check_model_last.h5

Epoch 00084: saving model to binary_model_double_float_initialised/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 85/100
592083/592083 [==============================] - 6s 10us/step - loss: 0.3206 - accuracy: 0.7192 - val_loss: 0.5948 - val_accuracy: 0.4652

***callbacks***
saving losses to binary_model_double_float_initialised/losses.log

Epoch 00085: val_loss did not improve from 0.32296

Epoch 00085: val_loss did not improve from 0.32296

Epoch 00085: saving model to binary_model_double_float_initialised/KERAS_check_model_last.h5

Epoch 00085: saving mode

592083/592083 [==============================] - 6s 10us/step - loss: 0.3209 - accuracy: 0.7189 - val_loss: 0.4070 - val_accuracy: 0.6767

***callbacks***
saving losses to binary_model_double_float_initialised/losses.log

Epoch 00099: val_loss did not improve from 0.32296

Epoch 00099: val_loss did not improve from 0.32296

Epoch 00099: saving model to binary_model_double_float_initialised/KERAS_check_model_last.h5

Epoch 00099: saving model to binary_model_double_float_initialised/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 100/100
592083/592083 [==============================] - 6s 10us/step - loss: 0.3210 - accuracy: 0.7187 - val_loss: 0.3916 - val_accuracy: 0.6550

***callbacks***
saving losses to binary_model_double_float_initialised/losses.log

Epoch 00100: val_loss did not improve from 0.32296

Epoch 00100: val_loss did not improve from 0.32296

Epoch 00100: saving model to binary_model_double_float_initialised/KERAS_check_model_last.h5

Epoch 00100: saving mod